# YAML component connections


We can define the netlist connections of a component by a netlist in YAML format

Note that you define the connections as `instance_source.port ->
instance_destination.port` so the order is important and therefore you can only
change the position of the `instance_destination`

For example, this coupler has the center coupling region at (100, 0)

In [ ]:
import pp

gap = 0.2
wg_width = 0.5
length = 10

yaml = f"""
instances:
    sl:
      component: coupler_symmetric
      settings:
        gap: {gap}
        wg_width: {wg_width}
    sr:
      component: coupler_symmetric
      settings:
        gap: {gap}
        wg_width: {wg_width}
    cs:
      component: coupler_straight
      settings:
        gap: {gap}
        width: {wg_width}
        length: {length}

placements:
    cs:
        x: 100
        y: 0

connections:
    sl,W0: cs,W0
    sr,W0: cs,E0

ports:
    w0: sl,E0
    w1: sl,E1
    e0: sr,E0
    e1: sr,E1

"""

c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

While this one has the sbend_left_coupler `sl` centered at (100, 0)

In [ ]:
gap = 0.2
wg_width = 0.5
length = 10

yaml = f"""
instances:
    sl:
      component: coupler_symmetric
      settings:
        gap: {gap}
        wg_width: {wg_width}
    sr:
      component: coupler_symmetric
      settings:
        gap: {gap}
        wg_width: {wg_width}
    cs:
      component: coupler_straight
      settings:
        gap: {gap}
        width: {wg_width}
        length: {length}

placements:
    sl:
        x: 100
        y: 0
        rotation: 180

connections:
    cs,W0: sl,W0
    sr,W0: cs,E0

ports:
    w0: sl,E0
    w1: sl,E1
    e0: sr,E0
    e1: sr,E1

"""

c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:        
    mmi_long:
        x: 100
        y: 100
"""


c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
placements:
    mmi_long:
        x: 100
        y: 100
routes:
    optical:
        mmi_short,E1: mmi_long,W0
"""


c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

You can **rotate** and instance specifying the angle in degrees

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
routes:
    optical:
        mmi_short,E1: mmi_long,E0
"""


c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

You can also define ports for the component

In [ ]:
import pp

yaml = """
instances:
    mmi_long:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 10
    mmi_short:
      component: mmi1x2
      settings:
        width_mmi: 4.5
        length_mmi: 5
        
placements:
    mmi_long:
        rotation: 180
        x: 100
        y: 100
        
routes:
    optical:
        mmi_short,E1: mmi_long,E0
    
ports:
    E0: mmi_short,W0
    W0: mmi_long,W0
"""

c = pp.component_from_yaml(yaml)
pp.show(c)
pp.plotgds(c)

In [ ]:
c.routes

In [ ]:
r = c.routes['mmi_short,E1:mmi_long,E0']
r

In [ ]:
r.parent.length

In [ ]:
c.instances

In [ ]:
c.routes

## Routes problem

As we saw in routing_bundles notebooks, for routing bundles of ports we need to use a bundle router

In [ ]:
import pp
sample_2x2_connections_problem = """
name:
    connections_2x2_problem

instances:
    mmi_bottom:
      component: mmi2x2
    mmi_top:
      component: mmi2x2

placements:
    mmi_top:
        x: 100
        y: 100

routes:
    optical:
        mmi_bottom,E0: mmi_top,W0
        mmi_bottom,E1: mmi_top,W1

"""


def test_connections_2x2_problem():
    c = pp.component_from_yaml(sample_2x2_connections_problem)
    return c

c = test_connections_2x2_problem()
pp.qp(c)
pp.show(c)

## Routes Solution

You can define several `bundle_routes` routed with a bundle router

In [ ]:
import pp

sample_2x2_connections_solution = """
name:
    connections_2x2_problem

instances:
    mmi_bottom:
      component: mmi2x2
    mmi_top:
      component: mmi2x2

placements:
    mmi_top:
        x: 100
        y: 100

routes:
    optical:
        mmi_bottom,E0: mmi_top,W0
        mmi_bottom,E1: mmi_top,W1

"""


def test_connections_2x2_solution():
    c = pp.component_from_yaml(sample_2x2_connections_solution)
    return c

c = test_connections_2x2_solution()
pp.qp(c)
pp.show(c)

In [ ]:
c.get_dependencies()

# Defining custom components

In [ ]:
@pp.autoname
def pad_new(size=(100, 100), layer=pp.LAYER.M3):
    c = pp.Component()
    compass = c << pp.c.compass(size=size, layer=layer)
    c.ports = compass.ports
    return c

c = pad_new(cache=False)
pp.qp(c)
pp.show(c)
print(c)

In [ ]:
import pp
from pp.components import component_factory

component_factory.update(pad_new=pad_new)
print('pad_new' in component_factory)

sample_custom_component = """
name:
    connections_2x2_problem

instances:
    bot:
      component: pad_new
    top:
      component: pad_new

placements:
    top:
        x: 0
        y: 200
"""


def test_sample_custom_component():
    c = pp.component_from_yaml(sample_custom_component, component_factory=component_factory)
    return c

c = test_sample_custom_component()
pp.qp(c)
pp.show(c)

# Defining custom routes

In [ ]:
import pp
from pp.components import component_factory
from pp.routing import route_factory

component_factory.update(pad_new=pad_new)
print('pad_new' in component_factory)


def connect_electrical_new(
    way_points= [],
    bend_factory = pp.c.corner,
    straight_factory= pp.c.wire,
    wg_width=10,
    bend_radius = 0.1,
    **kwargs,
):
    """ Returns a custom electrical route
    """
    bend90 = pp.call_if_func(bend_factory, radius=bend_radius, width=wg_width)
    connector = pp.routing.round_corners(way_points, bend90, straight_factory)
    return connector


route_factory.update(electrical_new=connect_electrical_new)


sample_custom_component = """
name:
    custom_routes

instances:
    bl:
      component: pad_new
    tl:
      component: pad_new
    br:
      component: pad_new
    tr:
      component: pad_new

placements:
    tl:
        x: 0
        y: 200

    br:
        x: 400
        y: 400

    tr:
        x: 400
        y: 600

routes:
    electrical_new:
        tl,E: tr,W
        bl,E: br,W
    optical:
        bl,S: br,E

"""


def test_sample_custom_routes():
    c = pp.component_from_yaml(sample_custom_component, component_factory=component_factory, route_factory=route_factory)
    return c

c = test_sample_custom_routes()
pp.qp(c)
pp.show(c)

In [ ]:
sample_custom_component = """
name:
    custom_routes

instances:
    t:
      component: pad_array
      settings:
          port_list: ['S']
    b:
      component: pad_array

placements:
    t:
        x: 200
        y: 400
routes:
    electrical:
        t,S0: b,N0
        t,S5: b,N5
"""


c = pp.component_from_yaml(sample_custom_component)
pp.qp(c)
pp.show(c)